在前一程式基礎上，算出每一類別可能導致成功之字詞

In [1]:
from sklearn import cluster, datasets
from sklearn.cluster import KMeans
import jieba
import csv
import numpy as np
punc = ["』","『","丨","｜","|","│","─","｜","－","—","〉","〈",">","<","&", "]", "[", "╱", "/", "0","０","1","１","2","２","3","３","4","４","5","５","6","６","7","７","8","８","9","９","。",".","；","=","●","%","【","】","《","》","：",":","+","?","？","!","！","，","," ,"「","　","」","＊","’",'"', " ","(",")","（","）","-","／","、"]
jieba.set_dictionary('C:\Program Files\Python36\lib\site-packages\jieba\dict.txt.big.txt')

In [2]:
stopwords = []
with open('stoplist.txt', 'r', encoding='utf8') as txtFile:
    for data in txtFile.readlines():
        data = data.strip()
        stopwords.append(data)

In [3]:
corpus = {}
dic = {}
count_row = 0
count_corpus = 0
temp_cors = []
with open("flyingV_money (1).csv", "r", encoding="utf16") as csvFile:
    rows = csv.DictReader(csvFile, delimiter = "\t")
    for row in rows:
        dic[count_row] = []
        temp_title = ""
        for temp_word in row["title"]:
            if temp_word in punc:
                continue
            temp_title += temp_word
        words = jieba.cut(temp_title)
        if row["money"] < row["targetmoney"]:
            continue
        for word in words:
            if word not in stopwords:
                dic[count_row].append(word)
                if word not in corpus:
                    temp_cors.append(word)
                    corpus[word] = 1
                    count_corpus += 1
                else: 
                    corpus[word] += 1
        count_row += 1

Building prefix dict from C:\Program Files\Python36\lib\site-packages\jieba\dict.txt.big.txt ...
Loading model from cache C:\Users\ucer\AppData\Local\Temp\jieba.ufdf3adf8ca174fd60fd39d3555294534.cache
Loading model cost 1.605 seconds.
Prefix dict has been built succesfully.


In [4]:
X = []

# Y = [0] * count_corpus
# X = []
# for row in range(count_row):
#     X.append(Y)


df_dic = {}
for word in temp_cors:
    temp = []
    for article in dic:
        if word in dic[article]:
            temp.append(article)
    df_dic[word] = temp

for word in df_dic:
    temp = []
    for article in dic:
        if word in dic[article]:
            temp.append(1)
        else:
            temp.append(0)
    X.append(temp)
X = np.array(X)

In [5]:
K = [3,5]

dis_vec = {}
# KMeans 演算法
final_X2 = {}
count_member = {}
for k in K:
    dis_vec[k] = {}
    X2 = {}
    temp = []
    print(k)
    with open(str(k) + "_result(success).txt", "r", encoding = "utf8") as txtFile:
        for row in txtFile:
            try:
                temp.append(int(txtFile.readline()[0]))
            except:
                continue
    # with open(str(k) + "_result(best).txt", "r", encoding = "utf8") as txtFile:
    #     for row in txtFile:
    #         try:
    #             temp.append(int(txtFile.readline()[0]))
    #             if temp[-1] not in dis_vec[k]:
    #                 dis_vec[k][temp[-1]] = []
    #             dis_vec[k][temp[-1]].append((final_X2[k][len(temp) - 1]))
    #             print(dic[len(temp) - 1])

    #         except:
    #             continue
    
    count_member[k] = []
    for i in range(k):
        count_member[k].append(temp.count(i))
    count = []
    for i in range(k):
        count.append(temp.count(i))
    for article in range(len(temp)):
        if temp[article] not in X2:
            X2[temp[article]] = {}
        for word in dic[article]:
            if word not in X2[temp[article]]:
                X2[temp[article]][word] = 1
            else:
                X2[temp[article]][word] += 1
    final_X2[k] = X2

3
5


In [7]:
for k in K:
    dis_vec[k] = {}
    temp = []
    print(k)
    with open(str(k) + "_result(success).txt", "r", encoding = "utf8") as txtFile:
        for row in txtFile:
            try:
                temp.append(int(txtFile.readline()[0]))
                if temp[-1] not in dis_vec[k]:
                    dis_vec[k][temp[-1]] = []
                dis_vec[k][temp[-1]].append((X[len(temp) - 1]))
                # print(X[len(temp) - 1])

            except:
                continue

3
5


In [8]:
count_vector = 0
distance = {}
for k in dis_vec:
    distance[k] = {}
    temp_arr = []
    for clus in dis_vec[k]:
        sum_array = 0
        for vector in dis_vec[k][clus]:
            # print(len(vector))
            temp_arr.append((vector))
            count_vector += 1
        mean_vector = np.array(temp_arr).mean(0)
        # print(mean_vector)
        for vector in dis_vec[k][clus]:
            temp = vector - mean_vector
            for i in temp:
                sum_array += i**2
        sum_array /= count_vector
        distance[k][clus] = sum_array
print(distance)

{3: {1: 4.074783102219263, 2: 0.9693154166924646, 0: 0.1163682348605019}, 5: {3: 1.6754971656073845, 0: 0.48218579640362663, 4: 0.18825307450530895, 2: 0.03154071360662013, 1: 0.0017633951847231188}}


In [9]:
final_score = {}
count_A = 0
for k in K:
    clus_score = {}
    for clus in range(k):
        score = {}
        for term in temp_cors:
            temp_score = 0
            ac = 0
            try:
                if term in final_X2[k][clus]:
                    ac = final_X2[k][clus][term]
            except:
                continue
            df = corpus[term]
            # for temp_class in range(k):
            #     if term in final_X2[k][temp_class]:
            #         df += final_X2[k][temp_class][term]
            #     if df == 0:
            #         print("AAAAAAAAAAAAA")
            #         count_A += 1
            ad = df - ac
            bc = count[clus] - ac

            bd = count_row - ad - ac - bc
            score[term] = temp_score
            a = ac + ad
            b = bc + bd
            c = ac + bc
            d = ad + bd
            T = a + b
            if ac>0:
                print(ac,ad,bc,bd, term)
                term_score = ((a*c)/T-ac)**2/((a*c)/T) + ((b*c/T-bc))**2/((b*c)/T) + ((a*d/T-ad))**2/((a*d)/T) + ((b*d/T-bd))**2/((b*d)/T)
                score[term] = term_score

        clus_score[clus] = score

    final_score[k] = clus_score

1 5 158 1546 自由
1 31 158 1520 生活
1 64 158 1487 世界
1 17 158 1534 畢業
1 32 158 1519 新
1 1 158 1550 好好
1 153 158 1398 台灣
1 1 158 1550 愛上
2 106 157 1445 計劃
1 15 158 1536 幸福
2 4 157 1547 無線
1 7 158 1544 充電
1 8 158 1543 書
1 10 158 1541 原創
1 7 158 1544 帶著
1 14 158 1537 走
1 0 158 1551 潮流
1 0 158 1551 ARK
1 0 158 1551 模組
3 326 156 1225 計畫
1 0 158 1551 Soyee
1 1 158 1550 點
1 0 158 1551 一盞燈
1 6 158 1545 回家
1 1 158 1550 感覺
2 143 157 1408 募資
1 6 158 1545 手作
1 5 158 1546 玩
1 39 158 1512 創作
1 6 158 1545 聲音
1 0 158 1551 Rangers
1 9 158 1542 單車
1 1 158 1550 遊俠
1 0 158 1551 包暮樂
1 0 158 1551 什物
1 0 158 1551 裝載
1 1 158 1550 騎士
1 20 158 1531 夢
1 8 158 1543 更
1 30 158 1521 看見
1 1 158 1550 筆記
1 13 158 1538 需要
1 21 158 1530 製作
1 8 158 1543 全台
1 14 158 1537 愛心
1 0 158 1551 東方
1 0 158 1551 舞茶
1 5 158 1546 聽見
1 0 158 1551 一杯
1 8 158 1543 美麗
1 5 158 1546 天然
1 0 158 1551 覓密
1 0 158 1551 八十年
1 0 158 1551 老店
1 5 158 1546 天
1 0 158 1551 德行
1 1 158 1550 旅遊
1 0 158 1551 ｛
1 0 158 1551 貝貝
1 0 158 1551 五十
1 0 158 1551 音
1

2 5 1 1702 基金會
1 0 2 1707 飛銳
2 0 1 1707 雨衣
3 7 0 1700 活動
2 0 1 1707 走私
2 0 1 1707 成功
1 0 2 1707 黑幫
1 0 2 1707 角色扮演
2 13 1 1694 桌遊
1 0 2 1707 MEXICO
1 0 2 1707 ARMOURCardholder
1 0 2 1707 暮樂
1 1 2 1706 優雅
1 0 2 1707 真皮
1 0 2 1707 卡套
3 2 0 1705 工具
2 1 1 1706 卡
1 0 2 1707 PentheonSemitone
1 0 2 1707 球型
1 0 2 1707 DOLCEGUSTO
4 1 -1 1706 專用
1 0 2 1707 重複使用
1 0 2 1707 膠囊
1 1 2 1706 活
1 0 2 1707 手印
1 0 2 1707 活字
5 4 -2 1703 說
9 20 -6 1687 故事
1 5 2 1702 玩
1 0 2 1707 LFP
1 0 2 1707 香料
3 0 0 1707 香水
3 1 0 1706 實驗室
5 7 -2 1700 創造
24 16 -21 1691 創作
1 0 2 1707 個人化
2 0 1 1707 發亮
2 2 1 1705 衣
1 0 2 1707 HunkMan
2 0 1 1707 家人
5 1 -2 1706 隨身
1 0 2 1707 保鏢
1 0 2 1707 汽車
1 0 2 1707 安全氣囊
3 3 0 1704 聽
1 0 2 1707 愛駒
1 0 2 1707 是否
1 0 2 1707 還缺
3 0 0 1707 馬鞍
2 1 1 1706 解決
1 0 2 1707 分類
2 0 1 1707 貨幣
1 0 2 1707 TWIGGY
1 0 2 1707 第三代
1 0 2 1707 四國
1 0 2 1707 Kalight
1 0 2 1707 萬
1 0 2 1707 花燈
1 0 2 1707 免綁
1 0 2 1707 鞋帶
1 0 2 1707 小發明
3 0 0 1707 方便
1 0 2 1707 鏡
1 0 2 1707 藍
1 0 2 1707 藍染
4 1 -1 1706 眼鏡
2 0 1 1

1 2 2 1705 身障
1 5 2 1702 棒球
1 0 2 1707 籌拍
1 0 2 1707 相愛
1 0 2 1707 七種
2 0 1 1707 南
1 0 2 1707 十字星
1 0 2 1707 天空
1 0 2 1707 澳洲
2 0 1 1707 打工
2 0 1 1707 度假
2 0 1 1707 同志電影
1 0 2 1707 犧牲
1 0 2 1707 旅愛很美
1 0 2 1707 無法
1 0 2 1707 觸摸
1 0 2 1707 時下
1 2 2 1705 暴力
2 0 1 1707 動畫電影
1 0 2 1707 桃蛙
1 0 2 1707 源記
1 1 2 1706 動物園
14 22 -11 1685 國際
1 4 2 1703 機會
1 0 2 1707 精工
1 1 2 1706 小子
1 0 2 1707 末代
1 0 2 1707 叛亂
1 0 2 1707 犯
1 0 2 1707 CIN
1 0 2 1707 É
1 0 2 1707 MAJET
1 0 2 1707 AIME
3 3 0 1704 數位
2 4 1 1703 贊助
1 3 2 1704 再現
1 0 2 1707 蟬的
1 0 2 1707 七年
1 0 2 1707 七日
1 0 2 1707 NEVERGIVEUP
3 0 0 1707 動畫短片
1 4 2 1703 臺
1 0 2 1707 戀念
1 0 2 1707 小姐
1 4 2 1703 反核
1 0 2 1707 WhenMomVisits
1 0 2 1707 加大
1 1 2 1706 喜劇
1 0 2 1707 夢見
2 1 1 1706 映演
1 0 2 1707 募資千人
2 0 1 1707 首映會
2 1 1 1706 快遞
1 0 2 1707 用場
1 0 2 1707 電影吧
1 0 2 1707 齊
1 1 2 1706 柏林
1 0 2 1707 露天
1 0 2 1707 縱行
2 1 1 1706 囝
3 2 0 1705 仔
1 0 2 1707 電影院線
1 0 2 1707 千甲
1 0 2 1707 張睿銓囡
1 0 2 1707 HEYKID
1 0 2 1707 錄影帶
1 5 2 1702 家庭
1 0 2 1707 輕喜劇
1

1 1 2 1706 英雄
1 0 2 1707 之嘆
1 0 2 1707 聖樂
1 1 2 1706 狂想
1 1 2 1706 管樂隊
1 1 2 1706 寒假
2 0 1 1707 巡迴演出
1 0 2 1707 電子音樂
1 3 2 1704 大師
1 0 2 1707 池
1 0 2 1707 田亮
1 0 2 1707 司
1 0 2 1707 RyojiIkeda
2 1 1 1706 重擊
1 0 2 1707 來台
3 7 0 1700 ・
1 2 2 1705 表演
1 0 2 1707 秋野芒
4 20 -1 1687 偏鄉
2 0 1 1707 戲劇
2 0 1 1707 綠島
1 1 2 1706 遇見
3 10 0 1697 ‧
1 0 2 1707 楊貴妃
1 1 2 1706 相
1 0 2 1707 和樂
2 0 1 1707 舞劇
1 0 2 1707 坊
1 3 2 1704 勇闖
1 3 2 1704 愛丁堡
1 4 2 1703 臺南
1 1 2 1706 代表隊
1 1 2 1706 錦標賽
1 0 2 1707 芭蕾舞團
1 0 2 1707 創團
1 0 2 1707 成大流
1 0 2 1707 舞社
1 0 2 1707 枉
1 1 2 1706 做夢
1 0 2 1707 佇逐
1 2 2 1705 精品
1 0 2 1707 GivingAwaytheTheatre
1 0 2 1707 小劇場
2 0 1 1707 演摩莎
7 4 -4 1703 劇團
1 3 2 1704 尼亞
1 0 2 1707 舅舅
1 0 2 1707 StagedReading
2 0 1 1707 讀劇
1 4 2 1703 憨兒
1 2 2 1705 藝術創作
1 0 2 1707 壞
1 0 2 1707 鞋子
7 7 -4 1700 劇場
1 0 2 1707 盡頭
1 0 2 1707 BestFriendsForever
1 3 2 1704 印象
1 0 2 1707 金字招牌
1 0 2 1707 店子
1 0 2 1707 之冶茶
1 0 2 1707 四九
1 0 2 1707 節外生枝
1 0 2 1707 千位
1 2 2 1705 清寒
1 0 2 1707 進
1 0 2 1707 金魚
1 0 2

2 8 20 1680 耳機
1 20 21 1668 打造
5 60 17 1628 世界
1 2 21 1686 景觀
1 2 21 1686 第七屆
1 10 21 1678 展
2 16 20 1672 畢業
2 31 20 1657 新
3 19 19 1669 旅行
1 1 21 1687 驚喜
1 6 21 1682 服裝
1 2 21 1686 乾淨
6 20 16 1668 ．
1 1 21 1687 喇叭
3 8 19 1680 手機
1 7 21 1681 畫
1 4 21 1684 站
2 7 20 1681 個人
1 2 21 1686 好幫手
1 5 21 1683 系列
1 2 21 1686 簡約
3 24 19 1664 臺灣
5 23 17 1665 x
15 139 7 1549 台灣
1 10 21 1678 學生
1 4 21 1684 首次
1 1 21 1687 三
1 3 21 1685 都市
1 2 21 1686 墊
3 11 19 1677 創意
1 2 21 1686 實驗
12 96 10 1592 計劃
1 1 21 1687 行走
2 22 20 1666 城市
1 0 21 1688 泛用
1 3 21 1685 型
2 3 20 1685 杯
1 3 21 1685 套
1 9 21 1679 簡單
2 14 20 1674 幸福
4 34 18 1654 行動
2 4 20 1684 無線
1 5 21 1683 多功能
2 2 20 1686 最佳
2 4 20 1684 方案
2 5 20 1683 記憶
1 10 21 1678 原創
1 11 21 1677 手工
1 7 21 1681 帶著
3 31 19 1657 一個
1 4 21 1684 師大
1 6 21 1682 專屬
1 1 21 1687 個性
1 3 21 1685 鍵盤
2 2 20 1686 音響
1 7 21 1681 吉他
1 3 21 1685 烏克
1 3 21 1685 麗麗
1 5 21 1683 時尚
1 10 21 1678 全球
1 4 21 1684 第一款
1 4 21 1684 首創
1 3 21 1685 愛的
1 5 21 1683 堅持
1 12 21 1676 發行
1 19 21 1

...

2 3 619 1086 貼紙
1 5 620 1084 環保
1 0 620 1089 減肥
1 0 620 1089 Rangers
6 4 615 1085 單車
1 1 620 1088 遊俠
1 0 620 1089 包暮樂
1 0 620 1089 什物
1 0 620 1089 裝載
1 1 620 1088 騎士
5 16 616 1073 夢
3 4 618 1085 派對
1 0 620 1089 穿出
1 0 620 1089 Style
2 2 619 1087 享受
1 0 620 1089 ZENLET
6 3 615 1086 唯一
1 1 620 1088 帶給
1 0 620 1089 POKE
1 0 620 1089 必備
3 0 618 1089 神器
2 2 619 1087 安全帽
2 1 619 1088 防水
1 0 620 1089 防塵套
3 6 618 1083 更
4 1 617 1088 全世界
10 21 611 1068 看見
1 0 620 1089 BRENDAxLVG
1 0 620 1089 兩分鐘
1 0 620 1089 縮時
4 1 617 1088 影片
1 0 620 1089 FFWD
1 0 620 1089 sothat
5 9 616 1080 '
1 0 620 1089 sme
2 3 619 1086 ×
1 0 620 1089 許匡匡衣
3 2 618 1087 更好
1 1 620 1088 上班
1 0 620 1089 途中
1 0 620 1089 CUBBISH
10 16 611 1073 一起
1 1 620 1088 出遊
1 2 620 1087 一雙
1 0 620 1089 鞋
1 1 620 1088 一輩子
1 0 620 1089 ILIMAIxBrenda
3 8 618 1081 女孩
1 3 620 1086 提案
1 0 620 1089 coplay
1 0 620 1089 衣櫃
1 0 620 1089 少
1 0 620 1089 這件
1 0 620 1089 針織
1 0 620 1089 串起
1 2 620 1087 人情味
3 1 618 1088 新銳
1 12 620 1077 力量
1 0 620 1089 B

1 0 620 1089 ShareYourLove
1 1 620 1088 羅
1 0 620 1089 晧
1 0 620 1089 耘
1 1 620 1088 浪
1 0 620 1089 SecondLanguage
1 0 620 1089 SecondBridge
1 0 620 1089 巡迴演唱
3 0 618 1089 政
1 0 620 1089 再給
1 2 620 1087 一點
3 2 618 1087 時間
3 1 618 1088 黃
1 0 620 1089 立
1 0 620 1089 綺
1 0 620 1089 不想
1 2 620 1087 成為
1 1 620 1088 一種
1 3 620 1086 大人
1 0 620 1089 加一
1 1 620 1088 夢踐
1 0 620 1089 古典
3 3 618 1086 跨界
1 0 620 1089 忽然
1 0 620 1089 一天
1 1 620 1088 離開
4 11 617 1078 台北
1 0 620 1089 鄭興
1 3 620 1086 福爾摩沙
1 2 620 1087 委託
3 5 618 1084 年度
1 0 620 1089 幻日
1 0 620 1089 成軍
1 1 620 1088 十五
2 0 619 1089 週年紀念
1 0 620 1089 日冕
1 0 620 1089 之典
1 0 620 1089 死亡
1 0 620 1089 絲竹空
1 0 620 1089 爵士樂團
1 0 620 1089 手牽手
3 3 618 1086 迷你
1 0 620 1089 Pianoboy
1 0 620 1089 鋼琴演奏
1 0 620 1089 門田英司
1 0 620 1089 范安婷
1 1 620 1088 回到
1 0 620 1089 原點
1 0 620 1089 錄音
1 3 620 1086 一次
1 3 620 1086 日記
1 0 620 1089 丘沁偉
1 2 620 1087 病患
1 0 620 1089 FLUX
1 2 620 1087 多元
1 0 620 1089 觀點
17 9 604 1080 演出
1 0 620 1089 酒桃
2 0 619 1089 も
1 0 62

2 0 619 1089 應用
1 0 620 1089 開發者
1 2 620 1087 年會
1 0 620 1089 MOPCON
1 0 620 1089 奇點
1 2 620 1087 講座
1 0 620 1089 首個
1 0 620 1089 登上
1 0 620 1089 Google
1 0 620 1089 臺灣人
1 1 620 1088 百萬
1 1 620 1088 募資大
1 0 620 1089 bitFlorinAPP
1 0 620 1089 國軍
1 0 620 1089 互助
1 0 620 1089 回報
1 0 620 1089 AppforiOS
1 0 620 1089 租出
1 0 620 1089 豐富
1 0 620 1089 租
1 1 620 1088 出租
2 1 619 1088 網
1 0 620 1089 無名
2 0 619 1089 搬家
1 0 620 1089 網誌
1 0 620 1089 備份
1 0 620 1089 FacebookQsearch
1 0 620 1089 最快
1 0 620 1089 臉書
1 1 620 1088 塗鴉
1 0 620 1089 牆
1 0 620 1089 搜尋
1 0 620 1089 引擎
1 0 620 1089 MINDPLARDefender
1 0 620 1089 干擾
1 0 620 1089 框
1 0 620 1089 ForiPhone
1 0 620 1089 FUJO
1 0 620 1089 遙控車
1 0 620 1089 MOCOCAR
1 3 620 1086 APP
1 0 620 1089 介面
1 0 620 1089 極限
1 5 620 1084 營
1 1 620 1088 食農
1 0 620 1089 新創
1 0 620 1089 Car
1 0 620 1089 sTalk
2 0 619 1089 訊息
2 0 619 1089 板
1 0 620 1089 CLAYWAY
1 0 620 1089 銀
1 0 620 1089 黏土
1 0 620 1089 RAMS
1 0 620 1089 足壓
1 0 620 1089 測量機
1 0 620 1089 足部
1 0 620 1089

2 0 619 1089 美術系
1 0 620 1089 饗
1 0 620 1089 高餐
1 0 620 1089 會展
1 0 620 1089 音樂界
1 0 620 1089 第三股
1 0 620 1089 勢力
2 1 619 1088 動漫
1 0 620 1089 最初
1 0 620 1089 裸
1 0 620 1089 ROD
2 0 619 1089 雙棲
1 0 620 1089 世上
1 0 620 1089 BPersona
1 0 620 1089 微小
1 1 620 1088 藝術家
1 1 620 1088 失明
1 0 620 1089 編舞家
1 0 620 1089 滯留
1 0 620 1089 翱翔
1 3 620 1086 論壇
1 0 620 1089 表演藝術
1 1 620 1088 今天
1 0 620 1089 絕對值
1 0 620 1089 水墨
1 0 620 1089 Re
1 3 620 1086 美好
1 0 620 1089 手繪
1 0 620 1089 老師傅
1 1 620 1088 逐夢
1 0 620 1089 北台
1 3 620 1086 畫展
1 0 620 1089 Liptal
1 0 620 1089 原夢計畫
1 0 620 1089 洪選
1 0 620 1089 琇
1 2 620 1087 舞動
1 0 620 1089 瑪莎
1 0 620 1089 葛
1 0 620 1089 蘭姆
1 0 620 1089 身耕計畫
1 0 620 1089 第二十八屆
1 0 620 1089 淡江大學
1 0 620 1089 金韶獎
1 0 620 1089 ←
1 0 620 1089 →
1 0 620 1089 OntheTrainNamedTime
1 0 620 1089 準點
1 0 620 1089 飆夢
1 0 620 1089 爵劇
1 0 620 1089 影色
2 2 619 1087 舞團
1 0 620 1089 阿修羅
1 0 620 1089 當原藝
1 0 620 1089 我動
1 0 620 1089 故我在
1 0 620 1089 領航
1 5 620 1084 世代
1 0 620 1089 夢夢
1 0 620 1089

1 2 49 1658 特展
1 1 49 1659 我家
1 0 49 1660 隻
1 3 49 1657 貓
1 0 49 1660 遠方
1 1 49 1659 祝福
1 0 49 1660 農家
1 0 49 1660 你家
1 0 49 1660 作夥來
1 0 49 1660 非營利
1 2 49 1658 農產
1 0 49 1660 公平交易
1 1 49 1659 牛奶
1 2 49 1658 救
1 1 49 1659 稻田
1 2 49 1658 餐桌
1 2 49 1658 Season
1 0 49 1660 暮野
1 0 49 1660 尋籟
1 0 49 1660 雞舍
1 0 49 1660 非籠
1 1 49 1659 飼養
1 2 49 1658 雞
1 0 49 1660 放牧
1 0 49 1660 建置
1 0 49 1660 慢
1 0 49 1660 飛天
1 0 49 1660 使園
1 0 49 1660 遊會
1 1 49 1659 尋寶
1 0 49 1660 葡萄樹
1 0 49 1660 餐
1 0 49 1660 酒會
1 0 49 1660 大台北
1 0 49 1660 千人
1 0 49 1660 良食


In [10]:
ss = []
# for k in final_score:
for clus in final_score[3]:
    for term in final_score[3][clus]:
        if final_score[3][clus][term] not in ss:
            ss.append(final_score[3][clus][term])
ss.sort()

In [11]:
for k in K:
    print(k)
    for clus in final_score[k]:
        temp = []
        print(clus, end = "\t")
        print(count_member[k][clus])

        for term in final_score[k][clus]:
            temp.append(final_score[k][clus][term])
            # if ss[-1] == final_score[k][clus][term]:
            #     print(term)
            # if ss[-2] == final_score[k][clus][term]:
            #     print(term)
            # if ss[-3] == final_score[k][clus][term]:
            #     print(term)
        temp.sort()
        temp.reverse()
        temp_term = []
        for i in range(len(temp)):
            for term in final_score[k][clus]:
                if final_score[k][clus][term] == temp[i]:
                    temp_term.append(term)
                if len(temp_term) > 4:
                    break
            if len(temp_term) > 4:
                print(temp_term)
                break

3
0	19
['計畫', '台灣', '募資', '潮流', 'ARK']
1	669
['計畫', '募資', '計劃', '專輯', '台灣']
2	167
['實現', '英語', '音樂', '專案', '偶像']
5
0	159
['實現', '英語', '專案', '偶像', '偽']
1	3
['尖石', '錦屏', '合唱團', '爆米', '爆遍']
2	22
['宿營', '趙', '宿營', '趙', '街舞']
3	621
['專輯', '畢業', '首張', '包', '充電']
4	50
['服飾', '控制', '炙', '彘', 'PickPigs']
